# Verkennen

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from tensorflow.keras.models import load_model

def get_layer(name, model):
    return next(filter(lambda layer: layer.name == name, model.layers))

def filter_title(pattern, movies):
    return movies[movies['title'].str.contains(pattern, False)]

def cos_sim(movie_id, weights):
    sims = np.squeeze(cosine_similarity([weights[movie_id]], weights))[1:]
    return (
        pd
            .DataFrame({'movie_id': range(1, len(sims) + 1), 'cos_sim': sims, 'cos_sim_abs': np.abs(sims)})
            .sort_values('cos_sim_abs', ascending=False)
            .drop(columns='cos_sim_abs')
    )

def euc_sim(movie_id, weights):
    sims = np.squeeze(euclidean_distances([weights[movie_id]], weights))[1:]
    return (
        pd
            .DataFrame({'movie_id': range(1, len(sims) + 1), 'euc_sim': sims})
            .sort_values('euc_sim')
    )

def movie_url(imdb_id):
    return f'https://www.imdb.com/title/tt{imdb_id:07}/'

def join_desc(sims, movies):
    ss = sims.merge(movies, 'left', 'movie_id')
    ss = (
        ss
            .drop(columns='imdb_id')
            .join(ss['imdb_id'].apply(movie_url))
    )
    sim_cols = list(sims.columns)[1:]
    ss = ss[['title', 'imdb_id'] + sim_cols]
    ss.columns = ['title', 'movie_url'] + sim_cols
    return ss

def top(sims, movies, n=5):
    return join_desc(sims, movies).head(n)

def count_ratings(movie_id, ratings):
    return len(ratings[ratings['movie_id'] == movie_id])

def drop_active(sims, ratings, z=3, z_left=None, z_right=None):
    ratings_movie = ratings.groupby('movie_id')['rating'].count()
    x, s = ratings_movie.mean(), ratings_movie.std()
    z_l = z_left or ((not z_right) and z) or None
    z_r = z_right or ((not z_left) and z) or None
    l = (z_l and x - s * z_l) or 1
    r = (z_r and x + s * z_r) or ratings_movie.max()
    active_ids = ratings_movie[np.logical_or(ratings_movie < l, ratings_movie > r)].index.values
    return sims[~sims['movie_id'].isin(active_ids)]

In [2]:
movies = pd.read_parquet('data/movies_active.parquet')
ratings = pd.read_parquet('data/ratings_active.parquet')
users = pd.read_parquet('data/users_active.parquet')

In [3]:
movies_cf_r     = load_model('data/movies_cf_r.h5')
movies_cf_r_1   = load_model('data/movies_cf_r_1.h5')
movies_cf_r_2   = load_model('data/movies_cf_r_2.h5')
movies_cf_r_d   = load_model('data/movies_cf_r_d.h5')
movies_cf_r_d_1 = load_model('data/movies_cf_r_d_1.h5')
movies_cf_r_d_2 = load_model('data/movies_cf_r_d_2.h5')
movies_cf_r_d_3 = load_model('data/movies_cf_r_d_3.h5')
movies_h_r      = load_model('data/movies_h_r.h5')
movies_h_r_1    = load_model('data/movies_h_r_1.h5')
movies_cf_c     = load_model('data/movies_cf_c.h5')
movies_cf_c_1   = load_model('data/movies_cf_c_1.h5')
movies_cf_c_2   = load_model('data/movies_cf_c_2.h5')
movies_cf_c_d   = load_model('data/movies_cf_c_d.h5')
movies_cf_c_d_1 = load_model('data/movies_cf_c_d_1.h5')
movies_cf_c_d_2 = load_model('data/movies_cf_c_d_2.h5')
movies_cf_c_d_3 = load_model('data/movies_cf_c_d_3.h5')
movies_h_c      = load_model('data/movies_h_c.h5')
movies_h_c_1    = load_model('data/movies_h_c_1.h5')

# Gelijkaardigheid

In [4]:
top_ten = lambda sims, n=10: top(sims, movies, n)

In [5]:
movie_weights = get_layer('movie_emb', movies_h_r).get_weights()[0]

In [6]:
ratings_movie = ratings.groupby('movie_id')['rating'].count().sort_values(ascending=False)
(
    pd
        .DataFrame({'movie_id': ratings_movie.index, 'n_ratings': ratings_movie.values})
        .merge(movies[['movie_id', 'title']], 'left', 'movie_id')
        .head()
)

,movie_id,n_ratings,title
0,17520,1751,Gravity
1,15299,1694,The Wolf of Wall Street
2,14641,1525,Interstellar
3,21636,1492,Gone Girl
4,18617,1417,The Revenant


In [7]:
ratings_movie.describe()

count    32917.000000
mean        19.511651
std         79.595181
min          1.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       1751.000000
Name: rating, dtype: float64

## The Wolf of Wall Street

### Cosine

In [8]:
w_sims = cos_sim(15299, movie_weights)
top_ten(w_sims)

,title,movie_url,cos_sim
0,The Wolf of Wall Street,https://www.imdb.com/title/tt0993846/,1.000000
1,Moi un noir,https://www.imdb.com/title/tt0051942/,0.877996
2,Side Street,https://www.imdb.com/title/tt0042960/,-0.867535
3,Il rosso segno della follia,https://www.imdb.com/title/tt0064904/,-0.859313
4,Alaska,https://www.imdb.com/title/tt2290397/,-0.855769
5,She Freak,https://www.imdb.com/title/tt0062259/,0.854065
6,Penguins of Madagascar,https://www.imdb.com/title/tt1911658/,-0.852820
7,Personal Effects,https://www.imdb.com/title/tt1138489/,0.852145
8,Not Another Happy Ending,https://www.imdb.com/title/tt1808339/,-0.851639
9,Jag älskar dig - En skilsmässokomedi,https://www.imdb.com/title/tt5183654/,-0.843099


In [9]:
top_ten(drop_active(w_sims, ratings, z_left=-1))

,title,movie_url,cos_sim
0,The Wolf of Wall Street,https://www.imdb.com/title/tt0993846/,1.000000
1,Penguins of Madagascar,https://www.imdb.com/title/tt1911658/,-0.852820
2,Grudge Match,https://www.imdb.com/title/tt1661382/,-0.838660
3,Primal Fear,https://www.imdb.com/title/tt0117381/,0.818428
4,Victoria,https://www.imdb.com/title/tt4226388/,0.811713
5,Hot Pursuit,https://www.imdb.com/title/tt2967224/,-0.810775
6,The Monuments Men,https://www.imdb.com/title/tt2177771/,-0.791241
7,La vie d'Adèle,https://www.imdb.com/title/tt2278871/,0.790811
8,Fracture,https://www.imdb.com/title/tt0488120/,-0.782860
9,Captain Fantastic,https://www.imdb.com/title/tt3553976/,0.782487


### Euclid

In [10]:
w_sims = euc_sim(15299, movie_weights)
top_ten(w_sims)

,title,movie_url,euc_sim
0,The Wolf of Wall Street,https://www.imdb.com/title/tt0993846/,0.000000
1,Moi un noir,https://www.imdb.com/title/tt0051942/,0.088416
2,She Freak,https://www.imdb.com/title/tt0062259/,0.097123
3,Tazza,https://www.imdb.com/title/tt0875025/,0.102846
4,Tepepa,https://www.imdb.com/title/tt0063679/,0.104483
5,Fred Armisen: Standup For Drummers,https://www.imdb.com/title/tt7924798/,0.104544
6,Personal Effects,https://www.imdb.com/title/tt1138489/,0.104981
7,Primal Fear,https://www.imdb.com/title/tt0117381/,0.106166
8,"Ali, Rabiaa et les autres",https://www.imdb.com/title/tt0284839/,0.107532
9,Gold,https://www.imdb.com/title/tt3134422/,0.108659


In [11]:
top_ten(drop_active(w_sims, ratings, z_left=-1))

,title,movie_url,euc_sim
0,The Wolf of Wall Street,https://www.imdb.com/title/tt0993846/,0.000000
1,Primal Fear,https://www.imdb.com/title/tt0117381/,0.106166
2,Captain Fantastic,https://www.imdb.com/title/tt3553976/,0.118123
3,Creed,https://www.imdb.com/title/tt3076658/,0.127514
4,Saving Private Ryan,https://www.imdb.com/title/tt0120815/,0.128827
5,Crash,https://www.imdb.com/title/tt0375679/,0.132733
6,Harry Potter and the Goblet of Fire,https://www.imdb.com/title/tt0330373/,0.134331
7,Beauty and the Beast,https://www.imdb.com/title/tt2771200/,0.134665
8,Vertigo,https://www.imdb.com/title/tt0052357/,0.135063
9,Man on Fire,https://www.imdb.com/title/tt0328107/,0.135879


## Mad Max

In [12]:
filter_title(r'mad max', movies)[['movie_id', 'title', 'year']]

,movie_id,title,year
5106,4789,Mad Max,1979
5473,5141,Mad Max 2,1981
6204,5830,Mad Max Beyond Thunderdome,1985
18347,17210,Mad Max: Fury Road,2015
21854,20396,Mad Max Renegade,2011


In [13]:
mm_ids = [4789, 5141, 5830, 17210]

### Cosine

In [14]:
mm_sims = cos_sim(mm_ids[0], movie_weights)
top_ten(mm_sims)

,title,movie_url,cos_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,1.000000
1,Angela's Christmas,https://www.imdb.com/title/tt7829544/,0.836928
2,Touching the Void,https://www.imdb.com/title/tt0379557/,-0.835519
3,Action,https://www.imdb.com/title/tt10955578/,-0.812888
4,Inseparable Bros,https://www.imdb.com/title/tt10237612/,0.810021
5,Deo pa-i-beu,https://www.imdb.com/title/tt3754322/,0.807856
6,Nobody's Baby,https://www.imdb.com/title/tt0215516/,0.801314
7,Zhi Sha,https://www.imdb.com/title/tt3297524/,0.797147
8,The Intruder,https://www.imdb.com/title/tt6722030/,-0.797126
9,Salvar Davasi,https://www.imdb.com/title/tt0494816/,0.795676


In [15]:
count_ratings(mm_sims['movie_id'].iloc[1], ratings)

2

In [16]:
count_ratings(mm_sims['movie_id'].iloc[2], ratings)

21

In [17]:
count_ratings(mm_sims['movie_id'].iloc[3], ratings)

1

In [18]:
join_desc(mm_sims[mm_sims['movie_id'].isin(mm_ids)], movies)

,title,movie_url,cos_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,1.000000
1,Mad Max: Fury Road,https://www.imdb.com/title/tt1392190/,0.605659
2,Mad Max 2,https://www.imdb.com/title/tt0082694/,0.514428
3,Mad Max Beyond Thunderdome,https://www.imdb.com/title/tt0089530/,-0.230859


In [19]:
top_ten(drop_active(mm_sims, ratings, z_left=-1))

,title,movie_url,cos_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,1.000000
1,Spider-Man: Far from Home,https://www.imdb.com/title/tt6320628/,0.790033
2,A Good Day to Die Hard,https://www.imdb.com/title/tt1606378/,-0.726477
3,Money Monster,https://www.imdb.com/title/tt2241351/,-0.716139
4,Finding Dory,https://www.imdb.com/title/tt2277860/,0.715493
5,Joy,https://www.imdb.com/title/tt2446980/,-0.703371
6,"I, Frankenstein",https://www.imdb.com/title/tt1418377/,-0.699300
7,300: Rise of an Empire,https://www.imdb.com/title/tt1253863/,0.696041
8,Shot Caller,https://www.imdb.com/title/tt4633690/,0.693308
9,Moonlight,https://www.imdb.com/title/tt4975722/,-0.675179


### Euclid

In [20]:
mm_sims = euc_sim(mm_ids[0], movie_weights)
top_ten(mm_sims)

,title,movie_url,euc_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,0.000000
1,Angela's Christmas,https://www.imdb.com/title/tt7829544/,0.091416
2,Deo pa-i-beu,https://www.imdb.com/title/tt3754322/,0.096930
3,Zhi Sha,https://www.imdb.com/title/tt3297524/,0.099262
4,Salvar Davasi,https://www.imdb.com/title/tt0494816/,0.100297
5,Gun No 6,https://www.imdb.com/title/tt9371780/,0.102673
6,Inseparable Bros,https://www.imdb.com/title/tt10237612/,0.103425
7,Chavez: Inside the Coup,https://www.imdb.com/title/tt0363510/,0.104359
8,Mourning Son,https://www.imdb.com/title/tt4916682/,0.105367
9,Nobody's Baby,https://www.imdb.com/title/tt0215516/,0.107022


In [21]:
join_desc(mm_sims[mm_sims['movie_id'].isin(mm_ids)], movies)

,title,movie_url,euc_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,0.000000
1,Mad Max: Fury Road,https://www.imdb.com/title/tt1392190/,0.197655
2,Mad Max 2,https://www.imdb.com/title/tt0082694/,0.242196
3,Mad Max Beyond Thunderdome,https://www.imdb.com/title/tt0089530/,0.312663


In [22]:
top_ten(drop_active(mm_sims, ratings, z_left=-1))

,title,movie_url,euc_sim
0,Mad Max,https://www.imdb.com/title/tt0079501/,0.000000
1,Shot Caller,https://www.imdb.com/title/tt4633690/,0.117897
2,300: Rise of an Empire,https://www.imdb.com/title/tt1253863/,0.118079
3,Spider-Man: Far from Home,https://www.imdb.com/title/tt6320628/,0.121099
4,Back to the Future Part III,https://www.imdb.com/title/tt0099088/,0.123482
5,Roma,https://www.imdb.com/title/tt6155172/,0.129116
6,Den of Thieves,https://www.imdb.com/title/tt1259528/,0.134477
7,Avengers: Age of Ultron,https://www.imdb.com/title/tt2395427/,0.137383
8,Bohemian Rhapsody,https://www.imdb.com/title/tt1727824/,0.137954
9,Kung Fu Panda 3,https://www.imdb.com/title/tt2267968/,0.139010


## Star Wars

In [23]:
filter_title(r'star wars', movies)[['movie_id', 'title', 'year']]

,movie_id,title,year
4864,4559,Star Wars,1977
5241,4918,Star Wars: Episode V - The Empire Strikes Back,1980
5861,5503,Star Wars: Episode VI - Return of the Jedi,1983
9751,9208,Star Wars: Episode I - The Phantom Menace,1999
9757,9214,Star Wars: Episode II - Attack of the Clones,2002
9758,9215,Star Wars: Episode III - Revenge of the Sith,2005
17244,16186,Star Wars: The Clone Wars,2008
24481,22726,Star Wars: Episode VII - The Force Awakens,2015
24619,22846,Star Wars: Episode VIII - The Last Jedi,2017
24620,22847,Star Wars: Episode IX - The Rise of Skywalker,2019


In [24]:
sw_ids = [4559, 4918, 5503, 9208, 9214, 9215, 22726, 22846, 22847]

### Cosine

In [25]:
sw_sims = cos_sim(sw_ids[0], movie_weights)
top_ten(sw_sims)

,title,movie_url,cos_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,1.000000
1,Copenhagen,https://www.imdb.com/title/tt2459156/,0.892273
2,Once Upon a Time in Mumbaai,https://www.imdb.com/title/tt1395054/,0.888382
3,Minions,https://www.imdb.com/title/tt2293640/,-0.885258
4,Hard Rain,https://www.imdb.com/title/tt0120696/,-0.884198
5,Shurayukihime,https://www.imdb.com/title/tt0158714/,0.882943
6,Down with Love,https://www.imdb.com/title/tt0309530/,0.882780
7,Rassayel El Bahr,https://www.imdb.com/title/tt1664814/,0.874781
8,Extinction,https://www.imdb.com/title/tt3201640/,-0.873010
9,La vida precoz y breve de Sabina Rivas,https://www.imdb.com/title/tt2062575/,-0.871793


In [26]:
join_desc(sw_sims[sw_sims['movie_id'].isin(sw_ids)], movies)

,title,movie_url,cos_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,1.000000
1,Star Wars: Episode II - Attack of the Clones,https://www.imdb.com/title/tt0121765/,-0.828438
2,Star Wars: Episode I - The Phantom Menace,https://www.imdb.com/title/tt0120915/,-0.749900
3,Star Wars: Episode VIII - The Last Jedi,https://www.imdb.com/title/tt2527336/,0.710628
4,Star Wars: Episode VII - The Force Awakens,https://www.imdb.com/title/tt2488496/,0.668356
5,Star Wars: Episode V - The Empire Strikes Back,https://www.imdb.com/title/tt0080684/,0.665899
6,Star Wars: Episode III - Revenge of the Sith,https://www.imdb.com/title/tt0121766/,0.605252
7,Star Wars: Episode VI - Return of the Jedi,https://www.imdb.com/title/tt0086190/,0.493293
8,Star Wars: Episode IX - The Rise of Skywalker,https://www.imdb.com/title/tt2527338/,-0.216327


In [27]:
top_ten(drop_active(sw_sims, ratings, z_left=-1))

,title,movie_url,cos_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,1.000000
1,Minions,https://www.imdb.com/title/tt2293640/,-0.885258
2,De rouille et d'os,https://www.imdb.com/title/tt2053425/,0.868648
3,Exodus: Gods and Kings,https://www.imdb.com/title/tt1528100/,-0.857776
4,Birds of Prey: And the Fantabulous Emancipatio...,https://www.imdb.com/title/tt7713068/,-0.855193
5,The Truman Show,https://www.imdb.com/title/tt0120382/,0.852087
6,Terminator 2: Judgment Day,https://www.imdb.com/title/tt0103064/,0.835955
7,Cloverfield,https://www.imdb.com/title/tt1060277/,0.833549
8,Star Wars: Episode II - Attack of the Clones,https://www.imdb.com/title/tt0121765/,-0.828438
9,Fantastic Mr. Fox,https://www.imdb.com/title/tt0432283/,0.826264


### Euclid

In [28]:
sw_sims = euc_sim(sw_ids[0], movie_weights)
top_ten(sw_sims)

,title,movie_url,euc_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,0.000000
1,Copenhagen,https://www.imdb.com/title/tt2459156/,0.075377
2,Once Upon a Time in Mumbaai,https://www.imdb.com/title/tt1395054/,0.078464
3,David and Lisa,https://www.imdb.com/title/tt0055892/,0.084724
4,Rassayel El Bahr,https://www.imdb.com/title/tt1664814/,0.084903
5,Shurayukihime,https://www.imdb.com/title/tt0158714/,0.088900
6,Down with Love,https://www.imdb.com/title/tt0309530/,0.090298
7,Boys on the Side,https://www.imdb.com/title/tt0112571/,0.090807
8,Too Late Blues,https://www.imdb.com/title/tt0055534/,0.091456
9,The Future of Food,https://www.imdb.com/title/tt0427276/,0.092383


In [29]:
join_desc(sw_sims[sw_sims['movie_id'].isin(sw_ids)], movies)

,title,movie_url,euc_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,0.000000
1,Star Wars: Episode III - Revenge of the Sith,https://www.imdb.com/title/tt0121766/,0.142474
2,Star Wars: Episode VI - Return of the Jedi,https://www.imdb.com/title/tt0086190/,0.171412
3,Star Wars: Episode V - The Empire Strikes Back,https://www.imdb.com/title/tt0080684/,0.174859
4,Star Wars: Episode VIII - The Last Jedi,https://www.imdb.com/title/tt2527336/,0.185094
5,Star Wars: Episode IX - The Rise of Skywalker,https://www.imdb.com/title/tt2527338/,0.222789
6,Star Wars: Episode VII - The Force Awakens,https://www.imdb.com/title/tt2488496/,0.340009
7,Star Wars: Episode II - Attack of the Clones,https://www.imdb.com/title/tt0121765/,0.347434
8,Star Wars: Episode I - The Phantom Menace,https://www.imdb.com/title/tt0120915/,0.431434


In [30]:
top_ten(drop_active(sw_sims, ratings, z_left=-1))

,title,movie_url,euc_sim
0,Star Wars,https://www.imdb.com/title/tt0076759/,0.000000
1,Hotel Mumbai,https://www.imdb.com/title/tt5461944/,0.108982
2,Anomalisa,https://www.imdb.com/title/tt2401878/,0.109589
3,The Truman Show,https://www.imdb.com/title/tt0120382/,0.110116
4,Green Room,https://www.imdb.com/title/tt4062536/,0.113040
5,Fantastic Mr. Fox,https://www.imdb.com/title/tt0432283/,0.114491
6,The Hateful Eight,https://www.imdb.com/title/tt3460252/,0.114584
7,Harry Potter and the Deathly Hallows: Part 2,https://www.imdb.com/title/tt1201607/,0.120756
8,Law Abiding Citizen,https://www.imdb.com/title/tt1197624/,0.121036
9,Limitless,https://www.imdb.com/title/tt1219289/,0.122105


## Alien

In [31]:
filter_title(r'alien', movies)[['movie_id', 'title', 'year']]

,movie_id,title,year
4418,4141,Alien Zone,1978
4724,4426,The Alien Factor,1978
5026,4713,Alien,1979
5190,4869,The Alien Dead,1980
6329,5949,Aliens,1986
6555,6165,Alien from L.A.,1988
6830,6428,Alien Nation,1988
6977,6571,My Stepmother Is an Alien,1988
7090,6679,Alienator,1990
7844,7395,Alien³,1992


In [32]:
filter_title(r'prometheus', movies)[['movie_id', 'title', 'year']]

,movie_id,title,year
18113,16994,The Prometheus Project,2010
18654,17493,Prometheus,2012


In [33]:
a_ids = [4713, 5949, 7395, 8743, 17493, 21855]

### Cosine

In [34]:
a_sims = cos_sim(a_ids[0], movie_weights)
top_ten(a_sims)

,title,movie_url,cos_sim
0,Alien,https://www.imdb.com/title/tt0078748/,1.000000
1,RoboCop 3,https://www.imdb.com/title/tt0107978/,-0.962705
2,Enter the Dragon,https://www.imdb.com/title/tt0070034/,0.952735
3,The Happening,https://www.imdb.com/title/tt0949731/,-0.952035
4,Downsizing,https://www.imdb.com/title/tt1389072/,-0.951144
5,The Bourne Ultimatum,https://www.imdb.com/title/tt0440963/,0.949935
6,Rim of the World,https://www.imdb.com/title/tt8179388/,-0.949248
7,Batman & Robin,https://www.imdb.com/title/tt0118688/,-0.948921
8,King Kong,https://www.imdb.com/title/tt0024216/,0.948441
9,Haider,https://www.imdb.com/title/tt3390572/,0.947865


In [35]:
join_desc(a_sims[a_sims['movie_id'].isin(a_ids)], movies)

,title,movie_url,cos_sim
0,Alien,https://www.imdb.com/title/tt0078748/,1.000000
1,Aliens,https://www.imdb.com/title/tt0090605/,0.791019
2,Alien: Covenant,https://www.imdb.com/title/tt2316204/,0.776472
3,Alien: Resurrection,https://www.imdb.com/title/tt0118583/,-0.454473
4,Alien³,https://www.imdb.com/title/tt0103644/,0.120532
5,Prometheus,https://www.imdb.com/title/tt1446714/,0.076686


In [36]:
top_ten(drop_active(a_sims, ratings, z_left=-1))

,title,movie_url,cos_sim
0,Alien,https://www.imdb.com/title/tt0078748/,1.000000
1,Downsizing,https://www.imdb.com/title/tt1389072/,-0.951144
2,The Bourne Ultimatum,https://www.imdb.com/title/tt0440963/,0.949935
3,Dark Phoenix,https://www.imdb.com/title/tt6565702/,-0.940788
4,El cuerpo,https://www.imdb.com/title/tt1937149/,0.940226
5,Easy A,https://www.imdb.com/title/tt1282140/,-0.936896
6,Star Trek Into Darkness,https://www.imdb.com/title/tt1408101/,0.933500
7,Men in Black: International,https://www.imdb.com/title/tt2283336/,-0.932277
8,The Dark Knight Rises,https://www.imdb.com/title/tt1345836/,0.932008
9,Independence Day: Resurgence,https://www.imdb.com/title/tt1628841/,-0.930873


### Euclid

In [37]:
a_sims = euc_sim(a_ids[0], movie_weights)
top_ten(a_sims)

,title,movie_url,euc_sim
0,Alien,https://www.imdb.com/title/tt0078748/,0.000000
1,King Kong,https://www.imdb.com/title/tt0024216/,0.144288
2,Haider,https://www.imdb.com/title/tt3390572/,0.146512
3,Nema-ye Nazdik,https://www.imdb.com/title/tt0100234/,0.161316
4,Blade,https://www.imdb.com/title/tt0120611/,0.162879
5,The Dark Knight Rises,https://www.imdb.com/title/tt1345836/,0.164957
6,Taare Zameen Par,https://www.imdb.com/title/tt0986264/,0.168692
7,Cowboy Bebop: Tengoku no tobira,https://www.imdb.com/title/tt0275277/,0.176872
8,Star Trek Into Darkness,https://www.imdb.com/title/tt1408101/,0.177475
9,Knockin' on Heaven's Door,https://www.imdb.com/title/tt0119472/,0.178032


In [38]:
join_desc(a_sims[a_sims['movie_id'].isin(a_ids)], movies)

,title,movie_url,euc_sim
0,Alien,https://www.imdb.com/title/tt0078748/,0.000000
1,Aliens,https://www.imdb.com/title/tt0090605/,0.277754
2,Alien: Covenant,https://www.imdb.com/title/tt2316204/,0.322233
3,Alien³,https://www.imdb.com/title/tt0103644/,0.456718
4,Prometheus,https://www.imdb.com/title/tt1446714/,0.464771
5,Alien: Resurrection,https://www.imdb.com/title/tt0118583/,0.584700


In [39]:
top_ten(drop_active(a_sims, ratings, z_left=-1))

,title,movie_url,euc_sim
0,Alien,https://www.imdb.com/title/tt0078748/,0.000000
1,The Dark Knight Rises,https://www.imdb.com/title/tt1345836/,0.164957
2,Star Trek Into Darkness,https://www.imdb.com/title/tt1408101/,0.177475
3,The Bourne Ultimatum,https://www.imdb.com/title/tt0440963/,0.178117
4,Star Wars: Episode VII - The Force Awakens,https://www.imdb.com/title/tt2488496/,0.186614
5,Contratiempo,https://www.imdb.com/title/tt4857264/,0.190096
6,The Incredibles,https://www.imdb.com/title/tt0317705/,0.202713
7,The Matrix,https://www.imdb.com/title/tt0133093/,0.203655
8,The Exorcist,https://www.imdb.com/title/tt0070047/,0.206783
9,Terminator 2: Judgment Day,https://www.imdb.com/title/tt0103064/,0.209838
